When making a geospatial plot in matplotlib, you can use [maplotlib-scalebar library](https://pypi.org/project/matplotlib-scalebar/) to add a scale bar.

In [ ]:
import geopandas as gpd
from matplotlib_scalebar.scalebar import ScaleBar

## Creating a ScaleBar object

The only required parameter for creating a ScaleBar object is *dx*. This is equal to a size of one pixel in real world. Value of this parameter depends on units of your CRS:

### Projected coordinate system (meters)
The easiest way to add a scale bar is using a projected coordinate system with meters as units. Just set `dx = 1`:

In [ ]:
nybb = gpd.read_file(gpd.datasets.get_path('nybb'))
nybb.to_crs(32619, inplace=True)  # Must be converted to a projected coordinate system

ax = nybb.plot()
ax.add_artist(ScaleBar(1))

### Geographic coordinate system (degrees)

With a geographic coordinate system with degrees as units, *dx* should be equal to a distance in meters of two points with the same latitude (Y coordinate) which are one full degree of longitude (X) apart. You can calculate this distance with an online calculator [(e.g. Great Circle calculator)](http://edwilliams.org/gccalc.htm) or in geopandas:

In [ ]:
from shapely.geometry.point import Point

nybb = gpd.read_file(gpd.datasets.get_path('nybb'))
nybb.to_crs(4326, inplace=True)  # Using WGS 84 (in degrees)

# Creating the points and calculating the distance
points = gpd.GeoSeries([Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326)
# Points are on the same latitude, one degree of longitude apart, with roughly the coordinates of NYC
points = points.to_crs(32619)  # Points are converted from WGS 84 to a projected system for
# the calculation

distance_meters = points[0].distance(points[1])
# The result slightly differs from Great Circle calculator (84,921 and 84,767 meters)
# but the difference is insignificant

ax = nybb.plot()  # The final plot uses WGS 84 again
ax.add_artist(ScaleBar(distance_meters))

## Using other units 

The default unit for *dx* is m (meter). You can change this unit with the *units* and *dimension* parameters. Here is a list of some possible *units* for various values of *dimension*:

| dimension | units |
| -----     |:-----:|
| si-length | km, m, cm, um|
| imperial-length |in, ft, yd, mi|
|si-length-reciprocal|1/m, 1/cm|
|angle|deg|

## Customization of the scale bar

In [ ]:
nybb = gpd.read_file(gpd.datasets.get_path('nybb')).to_crs(32619)
ax = nybb.plot()

# Position and layout
scale1 = ScaleBar(
dx=1, label='Scale 1',
    location='upper left',  # in relation to the whole plot
    label_loc='left', scale_loc='bottom'  # in relation to the line
)

# Color
scale2 = ScaleBar(
    dx=1, label='Scale 2', location='center', 
    color='#b32400', box_color='yellow',
    box_alpha=0.8  # Slightly transparent box
)

# Font and text formatting
scale3 = ScaleBar(
    dx=1, label='Scale 3',
    font_properties={'family':'serif', 'size': 'large'},  # For more information see the cell below
    scale_formatter=lambda value, unit: f'> {value} {unit} <'
)

ax.add_artist(scale1)
ax.add_artist(scale2)
ax.add_artist(scale3)

Note: Font is specified by six properties: *family*, *style*, *variant*, *stretch*, *weight*, *size* (and *math_fontfamily*). See [more](https://matplotlib.org/stable/api/font_manager_api.html#matplotlib.font_manager.FontProperties).

For more information about matplotlib-scalebar library, see its [PyPI](https://pypi.org/project/matplotlib-scalebar/) or [GitHub](https://github.com/ppinard/matplotlib-scalebar) page.